# Precipitation modelling for tropical cyclones

Based on the Python package CLIMADA, using a Tropical Cyclone Rain (TCR) model:
- Read the docs: https://github.com/CLIMADA-project/climada_petals/blob/6381a3c90dc9f1acd1e41c95f826d7dd7f623fff/climada_petals/hazard/tc_rainfield.py#L268
- Tutorial: https://climada-petals.readthedocs.io/en/stable/tutorial/climada_hazard_TCRain.html

- Read the docs on TCTracks: https://github.com/CLIMADA-project/climada_python/blob/main/climada/hazard/tc_tracks.py 

### For the historical tropical cyclone track database IBTrACS

- Link: https://www.ncei.noaa.gov/products/international-best-track-archive

Only the TC tracks from 1940 to 2024 are used in this notebook for consistency in the overall project. This can be easily modified. 

### Overview

This notebook follows the steps below: 
1. Preparations: loading packages and setting a standard for fonts/colour use
2. Generate centroids: create the points for which the modelling will take place
3. Load tracks: loading the TC tracks to be used
4. Model the hazard: modelling precipitation for all selected tracks, output to be saved

# 1. Preparations

### 1a. Load packages

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import time as timer
import os
import xarray as xr
from requests import HTTPError
import cdsapi

In [3]:
import climada
from climada.hazard import TCTracks
from climada.hazard import TropCyclone, Centroids
from climada_petals.hazard import TCRain

### 1b. Import functions out python file

In [5]:
import importlib
import TC_hazard_modelling

importlib.reload(TC_hazard_modelling)

# 2. Generate centroids
from TC_hazard_modelling import create_centroids_from_shapefiles
from TC_hazard_modelling import plot_all_grids_with_zoom

# 3. Load tracks
from TC_hazard_modelling import select_tracks
from TC_hazard_modelling import extract_selected_tracks

#4. Model the hazard
from TC_hazard_modelling import downloading_ERA5_CDSAPI
from TC_hazard_modelling import extract_ERA5_parameters
from TC_hazard_modelling import compute_precipitation

In [6]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

### 1c. Create color palette and choose font

In [9]:
my_colors = {
    "510brown": "#c6bc8b",
    "510darkblue": "#00214d",
    "510lightblue": "#d4e6ff",
    "510red": "#dd281f",
    "510lightred": "#ff7876", 
    "510purple": "#958dbe"
}

plt.rcParams["font.family"] = "Arial"

# 2. Generate centroids: a grid of points from the country shapefiles

### 2a. Create a list of centroids per country

In [19]:
centroids_list, shapefiles_list = create_centroids_from_shapefiles(folder_path = "0-level-Leeward++",      # Path to folder of shapefile(s) 
                                                                   resolution_km = 2,                      # Desired resolution for point grid (km)
                                                                   buffer_km = 0)                          # Optional buffer around the borders of a shapefile (km)

total_centroids = sum(c.coord.shape[0] for c in centroids_list)
print(f"Total number of centroids: {total_centroids}")

Created centroids for 14 shapefiles.
Total number of centroids: 3669


# 3. Load IBTrACS data

### 3a. Use TCTracks to import tracks from IBTrACS

In [22]:
tracks = TCTracks.from_ibtracs_netcdf(provider = 'usa',                    # 1 provider for consistent parameters
                                           basin="NA",                     # North Atlantic basin
                                           year_range = (1940, 2024),      # Same time period as ERA5 reanalysis dataset
                                           interpolate_missing = True)     # Interpolate for missing values
print("Number of tracks:", tracks.size)

2025-10-08 21:25:22,815 - climada.hazard.tc_tracks - WARNING - 280 storm events are discarded because no valid wind/pressure values have been found: 1940141N22289, 1940252N15313, 1940299N22288, 1941267N14300, 1942215N16275, ...
2025-10-08 21:25:22,826 - climada.hazard.tc_tracks - WARNING - 50 storm events are discarded because only one valid timestep has been found: 1940240N26295, 1940263N12279, 1940266N29311, 1940294N12281, 1942283N30289, ...
Number of tracks: 989


### 3b. Pre-select tracks passing the study area
A larger area than for the hazards wind speed and storm surge is used, since it is known that TC-related precipitation can occur over large distances. 

In [24]:
tracks_LeewardIslands = select_tracks(tracks = tracks,                           # Output of step 3a
                                      lon_min = -85, lon_max = -55,              # -85 to -55 degrees West
                                      lat_min = 10, lat_max = 25)                # 10 to 25 degrees North

print("Number of tracks:", tracks_LeewardIslands.size)

Number of tracks: 443


### 3c. Create a list of the storm IDs of the selected tracks and load tracks again

In [26]:
# Make list of tc's for each unique storm-id's
sid_list = [track.attrs.get('sid') for track in tracks_LeewardIslands.data]

# Retrieve track data again, now per storm going through the Leeward region
tr_ = {}

for sid in sid_list: 
    tr_[sid] = TCTracks.from_ibtracs_netcdf(storm_id = sid, provider = 'usa', basin="NA", interpolate_missing = True)
    tr_[sid].equal_timestep(time_step_h = 0.5)    # interpolate to 0.5-hourly steps

# 4. Model the hazard

### 4a. Background infomration on the Tropical Cyclone Rainfall model

Given a TCTracks instance, TCRain computes the precipitation rates (in mm/h) for each historical and/or synthetic track at every centroid and track position. The precipitation rates are then translated into total amounts of rainfall (in mm) by multiplying with each track's time step sizes and summation over the whole storm life time. TCRain inherits from Hazard and has an associated hazard type "TR". 

In addition to common along-track variables, e.g. the wind field, the TCR model requires four additional variables: 
- gridded topography (surface elevation)
- gridded drag coefficients (derived from surface roughness)
- along-track 600 hPa temperature at the storm centre
- along-track 850 hPa wind speeds averaged over an annulus of 200-500 km around the storm centre

The first two are provided within CLIMADA, but the temperature and wind speed variables need to be provided by the user. When using historical IBTrACS records, it is recommended to extract the variables from the ERA5 reanalysis. Providers of synthetic track sets based on climate models (e.g. CHAZ) can often provide both variables along with the track data. 

**Steps taken:**
1. Downloading ERA 5 data using csdapi for timesteps based on TCTrack
2. Preparing downloaded ERA5 parameters to correct format
3. Combine parameters and run TCrain: save results in CSV file

### 4b. Downloading ERA5 climate data
_Runs in +/- 6,5 hours_

An account is needed in the Copernicus Climate Data Store, and the CDSAPI needs to be installed. See link: https://cds.climate.copernicus.eu/how-to-api, on how to access download climate data. In the first run, you will need to approve the terms before it will work. 

Per storm, the days and times are extracted from the track (tr_[sid]). For those days and times, the ERA5 data is downloaded and saved to a NetCDF file called 'era5_data_{sid}.nc'. The same large bounding box as above is used as input. 

In [33]:
# start_time = timer.time()

# downloading_ERA5_CDSAPI(sid_list = sid_list,                                 # Output of step 3c
#                         tc_track_dict = tr_,                                 # Output of step 3c
#                         lon_min = -85, lon_max = -55,                        # -85 to -55 degrees West
#                         lat_min = 10, lat_max = 25,                          # 10 to 25 degrees North
#                         output_nc_dir = 'ERA5_data')                         # Output path (folder) as desired

# end_time = timer.time()
# print(f"Total calculation time: {end_time - start_time:.2f} seconds ({(end_time - start_time)/60:.2f} minutes)")

### 4c. Preparing downloaded ERA5 parameters to the correct format
_Runs in +/- 3 minutes_

In [35]:
start_time = timer.time()

extract_ERA5_parameters(sid_list = sid_list,                    # Output of step 3c
                        tc_track_dict = tr_,                    # Output of step 3c
                        input_nc_dir = 'ERA5_data',             # Equal to output_nc_dir of step 4b
                        output_csv_dir = 'ERA5_parameters')     # Output path (folder) as desired

end_time = timer.time()
print(f"Total calculation time: {end_time - start_time:.2f} seconds ({(end_time - start_time)/60:.2f} minutes)")

All results saved, in 443 files.
Total calculation time: 159.87 seconds (2.66 minutes)


### 4d. Combine parameters in TCTrack and run TCRain to obtain total precipitation amounts per point, over the duration of each storm
_Runs in +/- 30 minutes_

Additional input is required: an elevation map of the area of interest. It was found that the resolution of the elevation map highly affects the resulting precipitation amounts. A sensitivity analysis is recommended for your area of interest. In this notebook, a DEM is downscaled to a resolution of 5 km (method: average). 

In [37]:
start_time = timer.time()

# Additional input: a Digital Elevation Model
# If not specified, an SRTM-based topography at 0.1 degree resolution provided with CLIMADA is used.
elev_file = "SRTM15Plus/output_SRTM15Plus_-69_-58_12_21_filled_downscaled5km.tiff"

compute_precipitation(sid_list = sid_list,                                              # Output of step 3c
                      tc_track_dict = tr_,                                              # Output of step 3c
                      shapefiles_list = shapefiles_list,                                # Output of step 2a
                      centroids_list = centroids_list,                                  # Output of step 2a
                      input_csv_path = 'ERA5_parameters',                               # Equal to output_csv_path of step 4c
                      model_kwargs = {'wind_model': "H08",# ,                              # Choose wind model as desired, For consistency Holland (2008) is used
                                      'elevation_tif': elev_file},                      # Path to DEM file
                      output_csv_path = 'rain_output/IBTrACS_precipitation_all++_DEM5km.csv')

end_time = timer.time()
print(f"Total calculation time: {end_time - start_time:.2f} seconds ({(end_time - start_time)/60:.2f} minutes)")

2025-10-08 21:44:20,050 - climada.util.api_client - WARNING - there is no internet connection but the client has stored the results of this very request sometime in the past.
2025-10-08 21:44:21,440 - climada.util.api_client - WARNING - there is no internet connection but the client has stored the results of this very request sometime in the past.
Error for SID 1978220N13325 and ISO SXM: HTTPSConnectionPool(host='climada.ethz.ch', port=443): Max retries exceeded with url: /data-api/v2/dataset/?name=c_drag_500&status=package-data&limit=100000 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B190B6BFD0>, 'Connection to climada.ethz.ch timed out. (connect timeout=300)'))
2025-10-08 21:44:43,778 - climada.util.api_client - WARNING - there is no internet connection but the client has stored the results of this very request sometime in the past.
2025-10-08 21:44:44,917 - climada.util.api_client - WARNING - there is no internet connection but the client has

### 4e. Apply a threshold to keep storms that lead to extreme values

In [14]:
df = pd.read_csv('rain_output/IBTrACS_precipitation_all++_DEM5km.csv')

# Keep only storms that have at least surge levels a *at least one* point ≥ 0.2 m
sids_to_keep = df[df['prec_total_mm'] >= 50]['SID'].unique()
df_filtered = df[df['SID'].isin(sids_to_keep)]

df_filtered.to_csv('rain_output/IBTrACS_precipitation_all++_DEM5km_POT.csv')

### 4e. Optional: check output

In [16]:
df_filtered = pd.read_csv('rain_output/IBTrACS_precipitation_all++_DEM5km_POT.csv')
print(f"Total size of output pre-threshold: {df.size}")               # Should be equal to number of storms * number of centroids * number of columns
print(f"Number of rows in output pre-threshold: {df.shape[0]}")       # Should be equal to number of storms * number of centroids

print(f"Total size of output post-threshold: {df_filtered.size}")               # Should be equal to number of storms * number of centroids * number of columns
print(f"Number of rows in output post-threshold: {df_filtered.shape[0]}")       # Should be equal to number of storms * number of centroids


print(f"\nOriginal number of storms (unique SIDs): {df['SID'].nunique():,}")
print(f"Filtered number of storms (unique SIDs): {df_filtered['SID'].nunique():,}")
print(f"Storms removed: {df['SID'].nunique() - df_filtered['SID'].nunique():,}")


# Compute the maximum wind speed per storm
storm_max = df_filtered.groupby('SID')['prec_total_mm'].max().reset_index()
storm_max.rename(columns={'prec_total_mm': 'storm_max_prec_mm'}, inplace=True)

print("\nMaximum total precipitation amounts in mm per storm (summary):")
print(storm_max['storm_max_prec_mm'].describe())

print("\nTop 5 storms by maximum total precipitation amounts:")
print(storm_max.sort_values('storm_max_prec_mm', ascending=False).head())

Total size of output pre-threshold: 8114465
Number of rows in output pre-threshold: 1622893
Total size of output post-threshold: 5444628
Number of rows in output post-threshold: 907438

Original number of storms (unique SIDs): 443
Filtered number of storms (unique SIDs): 248
Storms removed: 195

Maximum total precipitation amounts in mm per storm (summary):
count     248.000000
mean      473.798959
std       519.106111
min        52.570345
25%       155.416104
50%       314.617105
75%       558.421710
max      5200.503688
Name: storm_max_prec_mm, dtype: float64

Top 5 storms by maximum total precipitation amounts:
               SID  storm_max_prec_mm
134  1999318N17278        5200.503688
215  2017260N12310        2310.901826
53   1967249N13303        2177.163142
102  1989254N13340        2118.260953
106  1990277N16301        1828.825461


In [25]:
# Map ISO codes to country names
country_names = {
    'AIA': 'Anguilla',
    'ATG': 'Antigua and Barbuda',
    'BLM': 'Saint Barthélemy',
    'DMA': 'Dominica',
    'GLP': 'Guadeloupe',
    'KNA': 'Saint Kitts and Nevis',
    'MAF': 'Saint Martin',
    'MSR': 'Montserrat',
    'MTQ': 'Martinique',
    'PRI': 'Puerto Rico',
    'SAB': 'Saba & Sint Eustatius',
    'SXM': 'Sint Maarten',
    'VGB': 'British Virgin Islands',
    'VIR': 'U.S. Virgin Islands'
}

# Prepare a list for summary rows
summary_rows = []

# Group by country and storm ID
for iso, group_country in df_filtered.groupby('ISO'):
    country = country_names.get(iso, iso)
    
    per_storm_means = []
    per_storm_maxima = []
    
    # Group by storm (SID)
    for sid, storm_group in group_country.groupby('SID'):
        storm_prec = storm_group['prec_total_mm'].values
        if len(storm_prec) == 0:
            continue
        
        mean_prec = storm_prec.mean()
        max_prec = storm_prec.max()
        
        per_storm_means.append(mean_prec)
        per_storm_maxima.append(max_prec)
    
    # Convert to Series for convenience
    per_storm_means = pd.Series(per_storm_means)
    per_storm_maxima = pd.Series(per_storm_maxima)
    
    # Only storms with precipitation > 0
    per_storm_means_nonzero = per_storm_means[per_storm_means > 0]
    
    row = {
        'Country': country,
        'Maximum of mean': round(per_storm_means.max(), 1),
        'Mean of mean': round(per_storm_means.mean(), 1),
        'Number of storms with precipitation': len(per_storm_means_nonzero),
        'Mean of mean for storms with precipitation': round(per_storm_means_nonzero.mean(), 1) if len(per_storm_means_nonzero) > 0 else 0.0,
        'Maximum of maximum': round(per_storm_maxima.max(), 1),
        'Mean of maximum': round(per_storm_maxima.mean(), 1)
    }
    
    summary_rows.append(row)

# Convert to DataFrame
df_summary = pd.DataFrame(summary_rows)

# Output LaTeX table
latex_table = df_summary.to_latex(index=False, escape=False, column_format='lXXXXXX')
print(latex_table)


\begin{tabular}{lXXXXXX}
\toprule
Country & Maximum of mean & Mean of mean & Number of storms with precipitation & Mean of mean for storms with precipitation & Maximum of maximum & Mean of maximum \\
\midrule
Anguilla & 757.200000 & 36.700000 & 221 & 41.200000 & 947.900000 & 45.300000 \\
Antigua and Barbuda & 488.800000 & 38.900000 & 235 & 40.900000 & 1124.400000 & 96.300000 \\
Saint Barthélemy & 1348.500000 & 40.200000 & 200 & 49.800000 & 1358.900000 & 41.500000 \\
Dominica & 878.100000 & 87.700000 & 218 & 99.800000 & 2177.200000 & 368.100000 \\
Guadeloupe & 699.600000 & 64.700000 & 231 & 69.500000 & 2322.700000 & 321.700000 \\
Saint Kitts and Nevis & 1306.300000 & 84.300000 & 240 & 87.100000 & 5200.500000 & 276.600000 \\
Saint Martin & 738.700000 & 41.600000 & 235 & 43.900000 & 1061.000000 & 66.700000 \\
Montserrat & 818.400000 & 83.200000 & 237 & 87.100000 & 2240.400000 & 268.400000 \\
Martinique & 445.300000 & 51.700000 & 209 & 61.300000 & 2278.500000 & 321.900000 \\
Puerto Rico & 